### Post Call transcription and analysis with Azure OpenAI 
This demo notebook shows how to perform analysis on a Swiss German audio interaction between two parties. Example use cases include the analysis of call center customer interactions. We provide the following capabilities:
- sentiment analysis
- general summarization of the call
- summarization of each speaker's remarks with speaker recognition

Before running the cells, create environment variables in a .env file for the openAI API key and endpoint and the speech API key. There are some packages that can be installed via ``pip install``. 

Check here for the official GitHub repo including other samples: [GitHub - Azure OpenAI Samples](https://github.com/Azure/azure-openai-samples/blob/main/use_cases/call_center/notebooks/call_center.ipynb)

In [ ]:
import azure.cognitiveservices.speech as speechsdk

from dotenv import load_dotenv
load_dotenv()

import json, os
import string
import time
import wave

import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

openai.api_type = "azure"
openai.api_key = os.getenv('OPENAI_API_KEY') 
openai.api_base = os.getenv('OPENAI_API_ENDPOINT') 
openai.api_version = '2023-05-15'

SPEECH_KEY = os.getenv("SPEECH_API_KEY")

In [ ]:
def recognize_speech_from_file(filename):
    # Set up the subscription info for the Speech Service:
    # Replace with your own subscription key and service region (e.g., "westus").
    speech_key = SPEECH_KEY
    service_region = "westeurope"

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language = "de-CH",audio_config=audio_config)

    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        # print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    # speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    # speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

In [ ]:
text = recognize_speech_from_file("name_of_your_audio_file.wav")
print(text)

### Create a Prompt for Sentiment Analysis

In [ ]:
prompt_sentiment = f"Detect whether the sentiment of the customer is positive or negative. Just say positive or negative.\n\n{' '.join(text)}"

deployment_name="name_of_your_openai_deployment"

openai.Completion.create(engine=deployment_name,prompt=prompt_sentiment,temperature=0,max_tokens=30,)["choices"][0]["text"].strip(" \n")

### Create a Prompt for Summarization

In [ ]:
prompt_summary = f"Summarize the contents of the customer call.\n\n{' '.join(text)}"

result = openai.Completion.create(engine=deployment_name,prompt=prompt_summary,temperature=0,max_tokens=300,)["choices"][0]["text"].strip(" \n")
print(result)

### Create a Prompt for Speaker Recognition

In [ ]:
prompt_test = f"Split the conversation by the two speakers and summarize what each speaker said.\n\n{' '.join(text)}"

result = openai.Completion.create(engine=deployment_name,prompt=prompt_test,temperature=0,max_tokens=600,)["choices"][0]["text"].strip(" \n")
print(result)